In [2]:
import torch
import matplotlib.pyplot as plt
import yousef_model as model
import Naicheng.res.process_data.process_output as pro_out
import Naicheng.res.process_data.process_raw_data as pr
import Naicheng.res.process_data.dire_and_coor as dc
from Naicheng.res.process_data.dataset import tensor_dataset
from torch.utils.data import DataLoader
import seaborn as sns
import numpy as np
import json

In [3]:
input_coordinate = pr.read_coordinate(16, '../../data/Coordinates.dat')
input_three_dirc = dc.coordinates_directions_three(input_coordinate)

$R^{2}_{g} = (r_{1}^{2} + r_{2}^{2} + r_{3}^{2} + ... r_{n}^{2})/n $
$R^{4}_{g} = (r_{1}^{4} + r_{2}^{4} + r_{3}^{4} + ... r_{n}^{4})/n $
$R^{6}_{g} = (r_{1}^{6} + r_{2}^{6} + r_{3}^{6} + ... r_{n}^{6})/n $

In [14]:
def rg2(coordinate):
    """
    :param coordinate: coordinates of single polymer
    :return:
    """
    x = np.zeros(16)
    y = np.zeros(16)
    for i in range(16):
        x[i] = coordinate[i][0]
        y[i] = coordinate[i][1]
    rg_2 = np.mean((x - np.mean(x)) ** 2 + (y - np.mean(y)) ** 2)
    rg_4 = np.mean(np.sqrt((x - np.mean(x)) ** 2 + (y - np.mean(y)) ** 2) ** 4)
    rg_6 = np.mean(np.sqrt((x - np.mean(x)) ** 2 + (y - np.mean(y)) ** 2) ** 6)
    return rg_2, rg_4, rg_6

In [15]:
rg_2 = np.zeros(100000)
rg_4 = np.zeros(100000)
rg_6 = np.zeros(100000)
for i in range(100000):
    rg_2[i], rg_4[i], rg_6[i] = rg2(input_coordinate[i])
print(f'rg2 is {np.mean(rg_2)}')
print(f'rg4 is {np.mean(rg_4)}')
print(f'rg6 is {np.mean(rg_6)}')

rg2 is 6.7946684375
rg4 is 82.47559951171876
rg6 is 1368.04750044487


In [16]:
def slice_directions(output):
    z_dim = np.shape(output)[0]
    direction_dim = np.shape(output)[1]
    assert direction_dim == 15, "the input dimension should be (x, 15)"
    test_dirc = np.zeros([z_dim,direction_dim,1])
    for i in range(z_dim):
        for j in range(direction_dim):
            test_dirc[i][j][0] = output[i][j]
    # round direction
    test_dirc = np.round(test_dirc * 2) / 2
    return test_dirc

In [17]:
epoch = 4000
name = f'models/yousef_gan_{epoch}.pth.tar'
gen = model.GeneratorNet()
check_point = torch.load(name)
gen.load_state_dict(check_point['gen_state_dict'])
# check the output
output_list = []
num_ite = 100000 // 8
output_direction = np.zeros([num_ite * 8, 15, 1])
for i in range(num_ite):
    noise = model.noise(8)
    output = gen(noise).data.numpy()
    output_direction[8 * i : 8 * (i+1)] = slice_directions(output)
output_coor = dc.direction_coordinate_three(output_direction)

In [19]:
output_rg2 = np.zeros(num_ite * 8)
output_rg4 = np.zeros(num_ite * 8)
output_rg6 = np.zeros(num_ite * 8)
for i in range(len(output_rg2)):
    output_rg2[i], output_rg4[i], output_rg6[i] = rg2(output_coor[i])
print (np.mean(output_rg2))
print (np.mean(output_rg4))
print (np.mean(output_rg6))

6.830464375
88.6277009765625
1602.2039971215058


In [21]:
print (np.mean(output_rg2) / np.mean(rg_2))
print (np.mean(output_rg4) / np.mean(rg_4))
print (np.mean(output_rg6) / np.mean(rg_6))




1.0052682390361303
1.074592988729589
1.1711610865854376
